<a href="https://colab.research.google.com/github/SooinJung/SLM_project_redits/blob/main/QLoRA2_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -U -q mediapipe peft datasets huggingface_hub transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requ

In [2]:
  import mediapipe as mp
  from datasets import load_dataset
  from peft import LoraConfig, get_peft_model
  from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
  from datasets import load_dataset
  from huggingface_hub import notebook_login
  from peft import LoraConfig, get_peft_model
  from transformers import AutoTokenizer, AutoModelForCausalLM
  from transformers import DataCollatorForLanguageModeling

In [3]:
notebook_login()

In [4]:
# 1. Dataset 로드
data = load_dataset("jojo0217/korean_safe_conversation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/26.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26979 [00:00<?, ? examples/s]

### 데이터셋 전처리
##Gemma-it dataset format

```
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model

```


In [14]:
# 'prompt' 필드 생성 함수
def format_instruction(example):

    # 추가 컨텍스트(input 필드)가 있는 경우
    if example['input'] and len(example['input']) > 0:
        text = f"""user\n{example["instruction"]}\n{example["input"]}\nmodel\n{example["output"]}"""
    # input 필드가 없는 경우
    else:
        text = f"""user\n{example["instruction"]}\nmodel\n{example["output"]}"""

    return {'prompt': text}

# 데이터셋의 prompt 필드를 업데이트
dataset = data.map(format_instruction)

In [6]:
!pip install -q -U peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.2 MB/s eta 0:00:00


In [7]:
!nvidia-smi

Sat Sep 21 07:24:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login

### 모델 로드

1. 양자화: A GPU is needed for quantization.

base 모델과 IT 모델(Instruction Tuned) 모델이 있는데, 두 모델의 주된 차이는 IT 모델이 대화 형식의 상호작용에 최적화되어 사용자의 의도를 더 정확하게 파악하여 보다 정제된 답변을 제공하기 때문에 it 버전의 모델을 호출한다.

## 관련 링크 첨부
https://devocean.sk.com/blog/techBoardDetail.do?ID=165703&boardType=techBlog

In [9]:
model_id = 'google/gemma-2-2b-it'

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [12]:
'''import torch
print(torch.cuda.is_available())  # True이면 GPU 사용 가능
print(torch.cuda.device_count())   # GPU 개수 확인'''

False
0


In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [11]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

Map:   0%|          | 0/26979 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 21583
    })
    test: Dataset({
        features: ['instruction', 'output', 'input', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 5396
    })
})

In [12]:
train_data = dataset["train"]
test_data = dataset["test"]

In [13]:
print(train_data[0])
print(test_data[0])

{'instruction': '요즘 교도소 시설 봤음? 개팔자가 상팔자더라.', 'output': '최근에 교도소 시설을 직접 볼 기회는 없었지만, 교도소 시설에 대한 여러 의견이 있을 수 있습니다. 좀 더 구체적인 질문이나 관심사를 말씀해주시면, 그에 관련된 정보를 제공해드리겠습니다.', 'input': '', 'prompt': 'user\n요즘 교도소 시설 봤음? 개팔자가 상팔자더라.\nmodel\n최근에 교도소 시설을 직접 볼 기회는 없었지만, 교도소 시설에 대한 여러 의견이 있을 수 있습니다. 좀 더 구체적인 질문이나 관심사를 말씀해주시면, 그에 관련된 정보를 제공해드리겠습니다.', 'input_ids': [2, 1645, 108, 237526, 243728, 75985, 236840, 237433, 27941, 238551, 235248, 247269, 238036, 235336, 49532, 241634, 184819, 36203, 241634, 236645, 238589, 236950, 235265, 108, 2516, 108, 239185, 239969, 236179, 75985, 236840, 237433, 27941, 238551, 236392, 143636, 240449, 181539, 28693, 237880, 236214, 56341, 238220, 95091, 235269, 75985, 236840, 237433, 27941, 238551, 236179, 71100, 197781, 23594, 241253, 235832, 21167, 236392, 22618, 55496, 235265, 235248, 243976, 70231, 49061, 238308, 85024, 160587, 237465, 180521, 55526, 239308, 236417, 236791, 72163, 244943, 237138, 237014, 236569, 237722, 235269, 20350, 236179, 187003, 238602, 104569, 2367

In [15]:
def get_completion(query: str, model, tokenizer):

  prompt_template = """user
  {query}

  model
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, max_new_tokens=256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

# Fine tuning 이전
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.", model=model, tokenizer=tokenizer)
print(result)

user
  건강을 유지하기 위한 세 가지 팁을 알려주세요.
  
  model
  **건강을 유지하기 위한 세 가지 팁:**

1. **균형 잡힌 식단:** 
    * 다양한 채소, 과일, 통곡물, 단백질을 섭취하여 영양소를 충분히 섭취하세요. 
    * 규칙적인 식사 시간을 유지하고, 과도한 섭취를 피하세요.
    * 규칙적인 운동과 함께, 건강한 식단은 몸의 건강을 유지하는 데 매우 중요합니다.

2. **규칙적인 운동:** 
    * 하루에 30분 이상의 규칙적인 운동을 하세요. 
    * 걷기, 달리기, 수영, 요가 등 다양한 운동 방법을 즐겨보세요. 
    * 운동은 몸의 건강을 유지하고, 스트레스 해소에도 도움이 됩니다.

3. **충분한 수면:** 
    * 7-8시간의 충분한 수면을 취하세요. 
    * 규칙적인 수면 시간을 유지


### LoRA Finetuning

In [16]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overf

Step,Training Loss
10,2.215900
20,2.031100
30,1.957800
40,1.793200
50,1.916100
60,1.965800
70,1.902300
80,1.813200
90,1.857900
100,1.792200


TrainOutput(global_step=100, training_loss=1.9245419692993164, metrics={'train_runtime': 146.8243, 'train_samples_per_second': 2.724, 'train_steps_per_second': 0.681, 'total_flos': 1270125714401280.0, 'train_loss': 1.9245419692993164, 'epoch': 0.018533104758374646})

In [17]:
# Fine tuning 이후
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  건강을 유지하기 위한 세 가지 팁을 알려주세요.
  
  model
  건강한 식단을 섭취하세요. 균형 잡힌 식단을 섭취하여 영양소를 충분히 섭취하세요. 
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  



In [18]:
# Fine tuning 이후
result = get_completion(query="어제 저녁에 뭐 먹었어?",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  어제 저녁에 뭐 먹었어?
  
  model
  저는 인공지능이기 때문에 식사를 먹지 않습니다. 하지만, 사람들이 좋아하는 음식을 알고 싶으시다면, 어떤 음식을 좋아하는지 알려주세요. 그런 다음, 저는 그 음식에 대한 정보를 제공할 수 있습니다.



In [21]:
# Fine tuning 이후
result = get_completion(query="제주도 여행지 하나 추천해주라!",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  제주도 여행지 하나 추천해주라!
  
  model
  제주도는 맛있는 음식, 멋진 자연, 그리고 즐거운 여행을 모두 만끽할 수 있는 곳입니다. 
  
  제주도에서 추천하는 여행지 중 하나는 **제주도 섬의 맛집**입니다. 
  
  제주도 섬에는 다양한 맛집들이 많이 있습니다. 
  
  * **제주도 맛집 1: 맛집 1**
  * **제주도 맛집 2: 맛집 2**
  
  제주도 섬의 맛집을 즐기면서 제주도의 맛을 만끽할 수 있습니다. 
  
  제주도 섬의 맛집을 찾으시면, 맛집 1과 맛집 2를 추천합니다. 
  
  제주도 섬의 맛집을 즐기면서 제주도의 맛을 만끽할 수 있습니다. 
  
  제주도 섬의 맛집을 찾으시면, 맛집 1과 맛


### 모델 저장

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
trainer.save_model('/content/drive/MyDrive/ColabNotebooks/산학 llm/checkpoint')  # 모델 저장
trainer.save_state()  # 상태 저장